In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
import joblib 


In [3]:
df = pd.read_csv('Mobile-Price-Prediction-cleaned_data.csv')

In [4]:
df.head()

Ratings  RAM    ROM  Mobile_Size  Primary_Cam  Selfi_Cam  Battery_Power  \
0      4.3  4.0  128.0         6.00           48       13.0           4000   
1      3.4  6.0   64.0         4.50           48       12.0           4000   
2      4.3  4.0    4.0         4.50           64       16.0           4000   
3      4.4  6.0   64.0         6.40           48       15.0           3800   
4      4.5  6.0  128.0         6.18           35       15.0           3800   

   Price  
0  24999  
1  15999  
2  15000  
3  18999  
4  18999

In [5]:
df.shape

(807, 8)

In [6]:
df.describe()

Ratings         RAM         ROM  Mobile_Size  Primary_Cam  \
count  807.000000  807.000000  807.000000   807.000000   807.000000   
mean     4.112639    5.941760   64.390335     5.620660    47.821561   
std      0.367830    2.056359   53.868626     3.957409    11.155916   
min      2.800000    0.000000    2.000000     2.000000     5.000000   
25%      3.900000    6.000000   32.000000     4.500000    48.000000   
50%      4.100000    6.000000   32.000000     4.770000    48.000000   
75%      4.400000    6.000000   64.000000     6.300000    48.000000   
max      4.800000   12.000000  256.000000    44.000000    64.000000   

        Selfi_Cam  Battery_Power          Price  
count  807.000000     807.000000     807.000000  
mean     8.868649    3278.859975   14269.167286  
std      4.547254     937.628801   23092.739983  
min      0.000000    1020.000000     479.000000  
25%      8.000000    3000.000000     984.000000  
50%      8.000000    3000.000000    1699.000000  
75%     12.000000    3800.000000   18994.500000  
max     23.000000    6000.000000  153000.000000

In [7]:
df

Ratings   RAM    ROM  Mobile_Size  Primary_Cam  Selfi_Cam  Battery_Power  \
0        4.3   4.0  128.0         6.00           48       13.0           4000   
1        3.4   6.0   64.0         4.50           48       12.0           4000   
2        4.3   4.0    4.0         4.50           64       16.0           4000   
3        4.4   6.0   64.0         6.40           48       15.0           3800   
4        4.5   6.0  128.0         6.18           35       15.0           3800   
..       ...   ...    ...          ...          ...        ...            ...   
802      3.8   6.0   32.0         4.54           48       12.0           2800   
803      4.1   8.0   64.0         4.54           64        8.0           2500   
804      4.4   3.0   32.0         6.20           48        1.0           3800   
805      3.7  10.0   32.0         4.50           64        8.0           3500   
806      3.5   6.0   32.0         4.50           64       15.0           1050   

     Price  
0    24999  
1    15999  
2    15000  
3    18999  
4    18999  
..     ...  
802   1299  
803   1390  
804   9790  
805    799  
806    799  

[807 rows x 8 columns]

In [9]:
'Ratings', 'RAM', 'ROM', 'Mobile_Size', 'Primary_Cam', 'Selfi_Cam', 'Battery_Power'label_encoders = {}
for column in [, 'Price']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

In [10]:
X = df[['Ratings', 'RAM', 'ROM', 'Mobile_Size', 'Primary_Cam', 'Selfi_Cam', 'Battery_Power']]
y = df['Price'] 

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [13]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

Mean Squared Error: 1707.9634205861255


In [14]:
joblib.dump(model, 'Price_Pred.pkl')
joblib.dump(label_encoders, 'label_encoders.pkl')

['label_encoders.pkl']

In [16]:
import gradio as gr
import pandas as pd
import joblib

# Load the trained model and label encoders
model = joblib.load('Price_Pred.pkl')
label_encoders = joblib.load('label_encoders.pkl')

def predict_transaction(Ratings, RAM, ROM, Mobile_Size, Primary_Cam, Selfi_Cam, Battery_Power):
    # Prepare the input data
    input_data = pd.DataFrame({
        'Ratings': [float(Ratings)],
        'RAM': [float(RAM)],
        'ROM': [float(ROM)],
        'Mobile_Size': [label_encoders['Mobile_Size'].transform([Mobile_Size])[0]],
        'Primary_Cam': [label_encoders['Primary_Cam'].transform([Primary_Cam])[0]],
        'Selfi_Cam': [label_encoders['Selfi_Cam'].transform([Selfi_Cam])[0]],
        'Battery_Power': [label_encoders['Battery_Power'].transform([Battery_Power])[0]]
    })
    
    # Make prediction using the loaded model
    prediction = model.predict(input_data)
    return prediction[0]

# Create the Gradio interface
iface = gr.Interface(
    fn=predict_transaction,
    inputs=[
        gr.Textbox(label='Ratings'),
        gr.Textbox(label='RAM'),
        gr.Textbox(label='ROM'),
        gr.Textbox(label='Mobile_Size'),
        gr.Textbox(label='Primary_Cam'),
        gr.Textbox(label='Selfi_Cam'),
        gr.Textbox(label='Battery_Power')
    ],
    outputs=gr.Textbox(label="Mobile Price Prediction"),
    title="Mobile Price Prediction",
    description="Mobile Price Prediction value based on its characteristics."
)

iface.launch()

/Users/HIMABINDU/Documents/Projects/internh/studysession/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
